In [1]:
import os

# imdb title conventions
https://contribute.imdb.com/updates/guide/title_formats

- Title (year)
- Drop it for: in quotes, (TV / V / VG )
- Drop it for anything with {{


In [1]:
def processTitle ( title ):
    valid = True
    if ( title.startswith('"')):
        valid = False
    elif '(V)' in title or '(TV)' in title or '(VG)' in title:
        valid = False
    elif ('}' in title ):
        valid = False
    else:
        titParts = title.split(')')
        title = titParts[0] + ')'
        
    return (valid, title)

print(processTitle( "\Dante\'s Domicile\" (2013) {The Hearse: Part 1 (#2.5)}  (as Dante Darrk)"))
print(processTitle("The 1996 Fox Rock 'N Roll Skating Championships (1996) (TV)"))
print(processTitle("At Night (2008)  (co-director)"))
print(processTitle("Freddie Goes to Washington (1993) {{SUSPENDED}}"))

(False, '\\Dante\'s Domicile" (2013) {The Hearse: Part 1 (#2.5)}  (as Dante Darrk)')
(False, "The 1996 Fox Rock 'N Roll Skating Championships (1996) (TV)")
(True, 'At Night (2008)')
(False, 'Freddie Goes to Washington (1993) {{SUSPENDED}}')


This is the directory where "directors.list" and "ratings.list" can be found. I downloaded them from ftp://ftp.funet.fi/pub/mirrors/ftp.imdb.com/pub/

See description at http://www.imdb.com/interfaces

In [2]:
fileDir = r"/Users/emanuele/OneDrive/data/imdb2017/"

# Directors 

What we're going to do it wrap it all in a big ol' for loop.

1.	Skip to line 236
2.	If line starts with character, copy name to buffer
3.	If line starts with three tabs, write intead (name, title)
4.	Write line

In [8]:
path = fileDir + "directors.list"
outputpath = fileDir + "cleandirectors.list"

directorDictionary = {}

with open( path, "r", encoding = "ISO-8859-1") as fo:
    with open(outputpath, "w", buffering = 1) as out:
        print("Name of file: ", fo.name)
        lineNo = 0
        outline = 0
        director = "zzzz"
        for line in fo:
            lineNo += 1
            if (lineNo < 236): 
                continue
            elif ("----------------------" in line):
                break
            elif ( len( line ) < 2): # The gap between movies
                continue
            elif (line.startswith ('\t')):
                (valid, title) = processTitle( line.strip() );
            else: 
                parts = line.split('\t')
                director = parts[0]
                (valid, title) = processTitle( parts[-1].rstrip());

            if( valid ):
                directorDictionary [ title ] = director
 #               print(director, "     ", title)        
                out.write( director + "\t" + title + "\n")
                outline += 1
                if( outline % 1000 == 0):
                    print (outline, director, "    ", title)
#            if ( outline > 100000):
#                break

Name of file:  /Users/emanuele/OneDrive/data/imdb2017/directors.list
1000 Abbey, Leslye      Big Fish Blues (2016)
2000 Abella, Cesar S.B.      Perlas (2004)
3000 Abreu, Jorge (I)      The Second Cup (2015)
4000 Acosta Esparza, Damián      Contrabando salvaje (1988)
5000 Adams, Sofia      Positive (2014/VI)
6000 Adlund, Ardalan      Brownfinger (2006)
7000 Agha, Moqeet      Bite Your Tongue (2017)
8000 Aguirre, Alberto (I)      Bajo la mano de Jesús (1973)
9000 Ahmed, Wakil      Koto Shopno Koto Asha (2017)
10000 Akalin, Nuri      72hrs the Hague (2017)
11000 Akre, John (I)      Minneapolis Beneath the Asphalt Part 7 (2017)
12000 Alain, Carolle      Des faiseurs d'images (1982)
13000 Albert-Sobrino, Miriam      A Historia dun Satélite (2017)
14000 Aldridge, Jacques      Forget Love (2018)
15000 Alexandre, Joseph F.      Shadow Play (2007)
16000 Alioto, Jimmy (I)      PSA: This Must Be Stopped (2011)
17000 Allen, Marc (IX)      A Real Man (2001)
18000 Almeida, Max      Double Dose of Te

158000 Chang, Joseph (VIII)      Homecoming: Ho'i Hou (2014)
159000 Chappelle, Aleta      Goosed (1999)
160000 Chase, Jennida (I)      Lollipop, Don't Be a Hero (2009)
161000 Chautard, Emile      Le mystère de la chambre jaune (1913)
162000 Chen, Christine (V)      The Night Mares (????)
163000 Cheng, Chun Kan      Die sha (1984)
164000 Cheung, Alfred      Hao men ye yan (1991)
165000 Chibber, Ashima      Mere Dad Ki Maruti (2013)
166000 Ching, Siu-Tung      Chai gong (1993)
167000 Cho, Kin-Nam      Chao ji wu di zhui nu zai 2 zhi gou zai xiong xin (1997)
168000 Choodu, Maria      A Distinguished Dissonance (2015)
169000 Chowdhury, Pratyush L.      Roilo Baki Char (2014)
170000 Christie, Al      Fruits and Flowers (1914)
171000 Chu, Yen-Ping      Bao gao ban zhang 5 (1996)
172000 Chytilová, Vera      T.G.M. - osvoboditel (1990)
173000 Cinquemani, Francesco (II)      Offstage: Lontano dal palco (2014)
174000 Claramunt, Victor      Empathy (2015/I)
175000 Clarke, Matt (XXI)      Take My 

320000 Fésös, András      A Börönd (1992)
321000 Gaffney, Stephen (III)      Lucid (2014/I)
322000 Galang, Romeo N.      Ibigay mo sa akin ang langit (1975)
323000 Gallagher, Michael J.      Unfinished Business (2014/IV)
324000 Galuppo, Gustavo      Sorry (2008/I)
325000 Gandhi, Mohan      SP Bhayankar (1983)
326000 Garaño, Jon      Loreak (2014)
327000 Garcia, Leonardo L.      Manila China Town (1978)
328000 García, Charlie (I)      Uno de cien (2015)
329000 Garisto, Aníbal      Los ojos de América (2014)
330000 Gartz, Juho      Rintamapiirtäjät (1990)
331000 Gates, Daric      Crook (2009)
332000 Gavala, Maria (I)      Peri erotos (1982)
333000 Gebrekidan, Yared      Red Runway (2013)
334000 Gemr, Meinkruf      Fidget (2017/II)
335000 George, Logan (II)      Caroline (2018)
336000 Germain, Jonathan (III)      The Last Trial (2017)
337000 Ghaderi, Iraj      Atash (1972)
338000 Ghosh, Narayan (III)      Sud Asol (2001)
339000 Gibbons, Stephen      Share the Silence (1988)
340000 Gigante

477000 Kavun, Andrey      Kandagar (2010)
478000 Kazan, Scott      The War on Love (2012)
479000 Keenan, Jp      Willing to Break (2014)
480000 Keller, Michael (VI)      Spooks (2011)
481000 Kelly, Randy (II)      Connecting (2010)
482000 Kennedy, Gary (VI)      Journey of the Lightning Stalker (2015)
483000 Kephart, William (III)      Future Holds Music Video (2013)
484000 Kesh, Saman      Placebo: Loud Like Love (2014)
485000 Khalil, Ronnie      The Meter Man of Le Moutrechon (2014)
486000 Kharroub, Farah Abou      Florentina (2015)
487000 Khrzhanovskiy, Andrey      I s vami snova ya (1980)
488000 Kikoïne, Gérard      Hôtel pour jeunes filles (1980)
489000 Kim, Hyo-cheon      Wonhan maethin samdoli (1970)
490000 Kim, So-young (IV)      Ha-neol-sack go-hyang (2000)
491000 Kimura, Susumu (II)      Dear Susan (2013)
492000 King, Jonathan (VII)      Under the Mountain (2009)
493000 Kinugasa, Teinosuke      Koi (1924)
494000 Kirkwood, James      The Mountain Girl (1915)
495000 Kittinger, 

638000 Mimica, Vatroslav      Poslije deset godina (1954)
639000 Mintz, Murray      Big Business (1997)
640000 Mirzaei, Gulistan      For Sardar: The Afghan Journalist (2016)
641000 Mitchell, Bruce (I)      Beyond the Smoke (1929)
642000 Mitrani, Rhonda L.      Cuba mia (2002)
643000 Mizrahi, Togo      El bahar (1935)
644000 Moede, Titus      The Last American Hobo (1967)
645000 Mohite, Girish (I)      Sarvanaam (2017)
646000 Molinar, Héctor (I)      Espejo retrovisor (2002)
647000 Monca, Georges      Le manuel du parfait gentleman (1908)
648000 Monnier, Christian      Fishing in the Moonlight (2016)
649000 Montes Bradley, Eduardo      Double Obsession (1992)
650000 Moon, H. Paul      R. Luke DuBois: Running Out of Time (2010)
651000 Moore, Thomas (XXXI)      Razor (2010)
652000 Morales, Lázaro      El ropaviejero (1994)
653000 Morel, Gaël      À toute vitesse (1996)
654000 Morgan, Joseph (VIII)      Global Community Day 2014: Episcopal School of Jacksonville (2014)
655000 Moritz, Rein

796000 Rist, Preben J.      Und das Blatt wendet sich (1916)
797000 Rivera, Ronnie (I)      El Sol Como un Gran Animal Oscuro (2015)
798000 Roa, Nolberto      Fragmentos del Alma (2009)
799000 Roberts, Alan (I)      Censorship U.S.A. (1971)
800000 Robertson, Mike (VII)      The Greatest Love Story Ever Told (2005)
801000 Robison, Arthur      Le père célibataire (1931)
802000 Rock, Stephen (V)      Dead Burger (2017)
803000 Rodrigues, Ramiro      Mais Leve que o Céu (2016)
804000 Rodríguez Górriz, Oscar      Doris (2016)
805000 Rogan, James (I)      El loco de la Catedral (2009)
806000 Rohrer, Dan      The Littlest Ones: A Dogumentary (2009)
807000 Roman, Carlos Etzio      Signals 2 (2013)
808000 Romero, Martín (I)      Futuro saqueado (2004)
809000 Roos, Jørgen (I)      Friluft (1959)
810000 Rose, Elizabeth (XV)      Androgyny Limitless Variety (2011)
811000 Rosenblatt, Jay (I)      A Pregnant Moment (2001)
812000 Ross, Dick (I)      The Restless Ones (1965)
813000 Rossif, Frédéric    

956000 Troell, Jan      Hurricane (1979)
957000 Truman, Jack      Phone Sex Grandma (2006/I)
958000 Tsiklauri, Tim      Crackheads (2013)
959000 Tual, Denise      Olivier Messiaen et les oiseaux (1973)
960000 Tumolva, Norman      Me, Sarah + Dad (2015)
961000 Turner, Jimmy (VI)      A Salesman of Death (2007)
962000 Tweel, Clay      Make Believe (2010)
963000 Tôjô, Shohei      Chôriki sentai Ohranger Vs Kakuranger (1996)
964000 Uher, Stefan      Penelopa (1978)
965000 Underwood, Ron (I)      Heart and Souls (1993)
966000 Uribe Ortiz, José Luis      Via carnis (2004)
967000 Utku, Ümit      Ali'yi gördüm Ali'yi (1967)
968000 Vaidya, Prem      Ganges (1973)
969000 Vale, Travers      The Meal Ticket (1914)
970000 Vallero, Gianluca      Finimondo (2000)
971000 van den Broek, Eric      The Making of the Revolution (2001)
972000 van Geffen, Marco      Allerzielen (2005)
973000 Van Sant, Gus      8 (2008)
974000 Vandermeer, Gordon      Backdoor to Hollywood 9 (1989)
975000 Varela, Nicolas     

In [9]:
print( len(directorDictionary))
print ( directorDictionary['Star Wars (1977)'] )
print( directorDictionary['The Usual Suspects (1995)'])

951702
Lucas, George (I)
Singer, Bryan


# Ratings

The columns are
`New  Distribution  Votes  Rank  Title`

Each line looks like
`      2000.0.004      56   6.4  "#BlackLove" (2015)`

We only care about VOTES, RANK, and TITLE
- Votes: column 17-25
- Rank:  column 27-31
- Title: starts at column 33 to [-1] (then there's a newline)


1. Skip to line 297 (we dont need the top 250 or the bottom 50)
2. Stop at the line that starts with "-----------"

In [12]:
path = fileDir + "ratings.list"
outputpath = fileDir + "cleanratings.list"
noDirPath = fileDir + "nodirector.list"


with open( path, "r", encoding = "ISO-8859-1") as fo:
    with open(outputpath, "w", buffering = 1) as out:
        with open(noDirPath, "w") as outerr:           
            print("Name of file: ", fo.name)
            out.write("raters\tscore\ttitle\tdirector\n")
            lineNo = 0
            outline = 0
            director = "zzzz"
            for line in fo:
                lineNo += 1
                if (lineNo < 297): 
                    continue
                elif ( len(line) < 2):
                    continue
                elif ("----------------------" in line):
                    break

                numRatings = int(line[17:25])
                avgRank = float( line[27:31])
                (valid, title) = processTitle(line[32:-1].lstrip())

                if( valid ):

                    ln =  str(numRatings) + "\t" + str(avgRank) + "\t" + title

                    if( title in directorDictionary):
                        director = directorDictionary[ title ]
                        ln += "\t" + director
                        out.write( ln  + "\n")
                    else:
                        outerr.write( ln  + "\n")

                    outline += 1
                    if( outline % 1000 == 0):
                        print( ln )


Name of file:  /Users/emanuele/OneDrive/data/imdb2017/ratings.list
518449	8.9	12 Angry Men (1957)	Lumet, Sidney
5	9.0	24/7 (2006)	Cowley, Shane (I)
15	6.1	5-tsu kazoereba kimi no yume (2014)	Yamato, Yûki
27	6.2	A Bird in the Bush (2008)	Fredianelli, Michael
13	6.7	A Few Screws Loose (2008)	McIntyre, Craig J.
50	5.9	A Man Betrayed (1936)	Auer, John H.
7	9.3	A Sci-Fi Tribute (2015)	Shevade, Shubham Sanjay
9	6.9	A Yank in Australia (1942)	Goulding, Alfred J.
5	5.6	Aasmaan Mahal (1965)	Abbas, Khwaja Ahmad
9	6.7	Acrobat (2013)	Menz, Eduardo (I)
17	8.1	Affairs with Bears (2002)	Majer, Jay
24	6.5	Ai (1955)	Zhu, Ji
8	4.4	Al-sayyid bulti (1967)	Saleh, Tewfik
6	5.8	Alison (2009)	Schattel, Paul
9	7.7	Alone Together (2015/II)	Pinger, Jacob
203	6.1	American Dad: The New CIA (2005)	MacFarlane, Seth
11	8.6	An Easy Grand (2003)	Shaftel, Ethan
15	7.3	Ang pamilyang hindi lumuluha (2017)	De Guzman, Mes
32415	5.8	Another 48 Hrs. (1990)	Hill, Walter (I)
67	8.1	Appu Chesi Pappu Koodu (1958)	Prasad, L.V. (I)

7	4.4	Nationale (2006)	Barbey, Alix
568	5.7	Nene Raju Nene Mantri (2017)	Teja (I)
1324	7.5	Ni liv (1957)	Skouen, Arne
79	6.8	Nikté (2009)	Arnaiz, Ricardo
6	5.0	No Thanks (2012)	M McAlarney, Paul
17	6.3	Northern Lights (2000)	O'Neill, Liam (I)
7	7.6	Nuit de la Glisse: Addicted to Life (2014)	Donard, Thierry
46	5.6	O Mundo Mágico dos Trapalhões (1981)	Tendler, Silvio
22	6.6	Occupation: Hollywood (2005)	Owens, Marques T.
5	8.0	Ojcu (2015)	Skaya, Diana
88	5.7	On the Marriage Broker Joke as Cited by Sigmund Freud in Wit and Its Relation to the Unconscious or Can the Avant-Garde Artist Be Wholed? (1977)	Land, Owen
27	8.7	One, Please (2008)	Speziani, Giulietta
13	6.2	Orfej (2013)	Smalcelj, Tina
170	6.0	Ottorzhenie (2011)	Lert, Vladimir
23	7.9	Où vas-tu Moshé? (2007)	Ben Jalloun, Hassan
12	8.0	Pandanti Kapuram (1972)	Deepak, Lakshmi
28	4.5	Parasuram (2003)	Arjun (II)
48	6.2	Pasteur (1935)	Rivers, Fernand
228	5.5	Pekka ja Pätkä miljonääreinä (1958)	Lohikoski, Armand
2034	6.3	Personal Best (1982

In [13]:
s = '      2000.0.004      56   6.4  "#BlackLove" (2015)'
print( int(s[17:25]))
print( float(s[27:31]))
print( s[33:-1])

56
6.4
#BlackLove" (2015


In [14]:
import sys
print( sys.stdout.encoding )

UTF-8
